In [ ]:
import os
import os.path
import spacy
import sklearn.model_selection
import numpy as np
import lime
from lime import discretize
from anchor import anchor_text
from tqdm import tqdm
nlp = spacy.load('C:\\Users\\EZEE\\Desktop\\schools\\capstone\\anchor and lime\\en_core_web_lg\\en_core_web_lg-2.2.5')

In [2]:
def load_polarity(path='C:\\Users\\EZEE\\Desktop\\schools\\capstone\\anchor and lime'):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(l)
    return data, labels

data, labels = load_polarity()

from sklearn.model_selection import train_test_split
x_tr, x_te, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42, shuffle = True)

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords as sw
import itertools
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import itertools
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('stopwords')

#This block define a function 'expandContractions' that expan the short hand variant to it's base form. for example "I'm" -> 'i am'.

#define the constraction dictionary
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

c_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return contraction_dict[match.group(0)]
    return c_re.sub(replace, text.lower())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EZEE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\EZEE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_sentence(text):
    clean_text = []
    for corpus in text:
        #print(corpus)
        if type(corpus) is bytes:
            corpus = corpus.decode('utf-8')
        #chnage word to lower case, we want to ignore capital or lower cases
        lower_doc = corpus.lower()
 
        #expan construction of words into it's full representation
        expand_doc = expandContractions(lower_doc)
        
        #delete punctuation
        clean_doc = re.sub(r'[^\w\s]','',expand_doc)
        
        #split sentence into words
        tokenized_doc1 = word_tokenize(clean_doc)
        
        sww = sw.words() 
        new_corpus = [w for w in tokenized_doc1 if not w in sww]
        clean_text.append(new_corpus)
    return clean_text

x_train = clean_sentence(x_tr)
x_test = clean_sentence(x_te)

In [5]:
clean_data = x_train + x_test
word_to_ix = {}
for sentence in clean_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())
word_list += ['unk']
word_to_ix['unk'] = len(word_to_ix)
word_to_ix['unk']

19387

In [6]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-50") 

EMBEDDING_DIM = 50

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

C:\Users\EZEE\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


(19388, 50)

In [7]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(x_train,word_to_ix)
train_output_index = y_train
val_input_index = to_index(x_test,word_to_ix)
val_output_index = y_test

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class SelfAttention(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length):
        super(SelfAttention, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weight.data.copy_(torch.from_numpy(embedding_matrix))

        self.dropout = 0.3
        self.bilstm = nn.LSTM(embedding_length, hidden_size, dropout=self.dropout, bidirectional=True)
        # We will use da = 350, r = 30 & penalization_coeff = 1 as per given in the self-attention original ICLR paper
        self.W_s1 = nn.Linear(2*hidden_size, 350)
        self.W_s2 = nn.Linear(350, 30)
        self.fc_layer = nn.Linear(30*2*hidden_size, 2000)
        self.label = nn.Linear(2000, output_size)
  
    def attention_net(self, lstm_output):
        attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
        attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
        attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)
        return attn_weight_matrix

    def forward(self, input_sentences, batch_size=None):
        input = self.word_embeddings(input_sentences).view(len(input_sentences), 1, -1)
        if batch_size is None:
    
            h_0 = Variable(torch.zeros(2, 1, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, 1, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())

        output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
        output = output.permute(1, 0, 2)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        # h_n.size() = (1, batch_size, hidden_size)
        # c_n.size() = (1, batch_size, hidden_size)
        attn_weight_matrix = self.attention_net(output)
        #print(attn_weight_matrix)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        hidden_matrix = torch.bmm(attn_weight_matrix, output)
        # hidden_matrix.size() = (batch_size, r, 2*hidden_size)
        # Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
        fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
        logits = self.label(fc_out)
        x = F.log_softmax(logits, dim=1)

        return x, attn_weight_matrix



In [9]:
import torch.optim as optim
batch_size = None
output_size = 2
hidden_size = 128
vocab_size = len(word_to_ix)
embedding_length = 50
learning_rate = 0.001
epochs = 20
model = SelfAttention(batch_size, output_size, hidden_size, vocab_size, embedding_length).to('cuda')

device = torch.device("cuda")
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [10]:
m = len(train_input_index)
for epoch in range(epochs):  
    losses = []
    model.train()
    indexes = list(np.random.permutation(m))
    for i in tqdm(indexes, position = 0):
        tags_index = [train_output_index[i]]
        optimizer.zero_grad()
        sentence_in = torch.tensor(train_input_index[i], dtype=torch.long).to('cuda')
        #print(sentence_in.shape)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        output, _ = model(sentence_in)  
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    losses = np.array(losses)
    print('epoch:', epoch, '    loss:', np.mean(losses))
    print()

  0%|          | 0/6990 [00:00<?, ?it/s]C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
  0%|          | 8/6990 [00:00<01:33, 74.97it/s]

epoch: 0     loss: 0.628729089126737



  0%|          | 8/6990 [00:00<01:30, 77.13it/s]

epoch: 1     loss: 0.38734706294042015



  0%|          | 8/6990 [00:00<01:34, 74.27it/s]

...

epoch: 17     loss: 2.015253504970043e-05



  0%|          | 7/6990 [00:00<01:44, 66.84it/s]

epoch: 18     loss: 7.56868135946162e-08



100%|██████████| 6990/6990 [01:44<00:00, 66.61it/s]

epoch: 19     loss: 3.956588723287733e-08



In [11]:
correct = 0
correctPre = []
for i in range(len(val_input_index)):
    model.eval()
    
    
    sentence_in = torch.tensor(val_input_index[i], dtype=torch.long).to('cuda')
    output, weights = model(sentence_in) 
    pred = torch.argmax(output, axis = -1).cpu().numpy()[0]
    if pred == val_output_index[i]:
        correctPre.append(i)
        correct += 1
print(correct/len(val_input_index))

0.7232065059541097


# Affin

In [12]:
!pip install afinn
from afinn import Afinn
afinn = Afinn()

In [13]:
def afi(text, target):
    dic = {}
    ans = []
    for w in x_test[idx]:  
        dic[w] = afinn.score(w)
    if target == 0:
        for w in dic:
            if dic[w] <= -1: ans.append(w)
           
    if target == 1:
        for w in dic:
            if dic[w] >= 1: ans.append(w)
    return ans

In [16]:
indexes = []
for idx in correctPre:
    word = afi(x_test[idx], y_test[idx])  
    if len(word) > 0:
        indexes.append(idx)

In [17]:
affin = []
for i in indexes[0:50]:
    idx = i
    word = afi(x_test[idx], y_test[idx])  
    #print(word)
    affin.append(word)

# use self-attention

In [19]:
index = 18

model.eval()
sentence_in = torch.tensor(val_input_index[index], dtype=torch.long).to('cuda')
output, weights = model(sentence_in) 
avg_weight = torch.mean(weights, axis = 1)
idx = torch.topk(avg_weight, k = 3)
idx = idx.indices.cpu().numpy().tolist()[0]
word = [x_test[index][k] for k in idx]
pred = torch.argmax(output, axis = -1).cpu().numpy()[0]
print("true label:", y_test[index])
print('predicted label:', pred)
print('important features:', word)
print('original text:', x_te[index])
print(y_test[index])

true label: 1
predicted label: 0
important features: ['lame', 'art', 'adaptation']
original text: b"it ain't art , by a long shot , but unlike last year's lame musketeer , this dumas adaptation entertains ."
1


In [20]:
def selfatt(index):
    model.eval()
    sentence_in = torch.tensor(val_input_index[index], dtype=torch.long).to('cuda')
    output, weights = model(sentence_in) 
    avg_weight = torch.mean(weights, axis = 1)
    k = min(sentence_in.shape[0], 5)
    idx = torch.topk(avg_weight, k = k)
    idx = idx.indices.cpu().numpy().tolist()[0]
    word = [x_test[index][k] for k in idx]
    return word

self_attention = []
for index in indexes[0:50]:
    words = selfatt(index)
    self_attention.append(words)

# use anchor

In [21]:
def predict_nn(text):
    text = clean_sentence(text)
    text = to_index(text,word_to_ix)[0]
    model.eval()
    sentence_in = torch.tensor(text, dtype=torch.long).to('cuda')
    output, weights = model(sentence_in) 
    pred = torch.argmax(output, axis = -1).cpu().numpy()
    return pred


In [22]:
def find_anchor(index):
    explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=True)
    np.random.seed(1)
    text = x_te[index]
    pred = explainer.class_names[predict_nn([text])[0]]
    alternative =  explainer.class_names[1 - predict_nn([text])[0]]
    html = clean_sentence([text])[0]
    html = u' '.join(html)
    html
    exp = explainer.explain_instance(html, predict_nn, threshold=0.995)
    return exp.names()

anchor = []
for index in tqdm(indexes[0:50], position = 0):
    word = find_anchor(index)
    anchor.append(word)

100%|██████████| 50/50 [58:53<00:00, 70.67s/it]   


# use LIME

In [23]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [24]:
def findprob(text):
    text = clean_sentence(text)
    prob = []
    for i in range(len(text)):
        corpus = [word_to_ix[w] for w in text[i]]
        sentence_in = torch.tensor([19387]+corpus, dtype=torch.long).to('cuda')
        
        model.eval()
        out, _ = model(sentence_in) 
        out = out.cpu().detach().numpy()
        exp = np.exp(out)
        expsum = 0
        for i in range(len(exp[0])):
            expsum += exp[0][i]
        ans = exp/expsum
        prob.append(ans[0])
    return np.array(prob)

#findprob([ x_te[6], x_te[7] ])

In [25]:
class_names = ['neg', "pos"]
explainer = LimeTextExplainer(class_names=class_names)

In [28]:
def findlime(idx):
    text = x_te[idx].decode('utf-8')
    text = clean_sentence([text])[0]
    html = ' '.join(text)
    exp = explainer.explain_instance(html ,findprob, num_features=3, top_labels = 1)
    label = exp.top_labels[0]
    A = [u for (u, v) in exp.local_exp[label]]
    B = [text[u] for u in A]
    return B

lime = []
for index in indexes[0:50]:
    word = findlime(index)
    lime.append(word)

C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1


C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1


C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


...

0


C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1


C:\Users\EZEE\anaconda3\lib\site-packages\torch\nn\functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0


# recall

In [29]:
def recall(true, pred):
    correct = 0
    all = 0
    for i in range(len(pred)):
        all += len(true[i])
        for word in pred[i]:
            if word in true[i]: correct += 1
    return correct/all

print('lime:', recall(self_attention, lime))
print('anchor:', recall(self_attention, anchor))

lime: 0.41735537190082644
anchor: 0.5330578512396694


In [31]:
print('lime:', recall(affin, lime))
print('anchor:', recall(affin, anchor))

lime: 0.39473684210526316
anchor: 0.6052631578947368


# precision

In [32]:
def precision(true, pred):
    correct = 0
    all = 0
    for i in range(len(pred)):
        all += len(pred[i])
        for word in pred[i]:
            if word in true[i]: correct += 1
    return correct/all

print('lime:', precision(self_attention, lime))
print('anchor:', precision(self_attention, anchor))

lime: 0.6778523489932886
anchor: 0.524390243902439


In [34]:
print('lime:', precision(affin, lime))
print('anchor:', precision(affin, anchor))

lime: 0.20134228187919462
anchor: 0.18699186991869918
